Из набора данных варианта сформировать два массива данных: 
- массив количественных данных (все количественные данные оставить без изменений, для всех качественных данных кроме переменной класса (переменная класса – переменная A16 для вариантов 1-10 (+ положительный класс, - отрицательный)) произвести калибровку с учетом априорного шанса с поправкой Лапласа, значения переменной класса задать как 1, если положительный класс, 0 иначе); 
- массив качественных данных (для всех качественных данных кроме переменной класса произвести калибровку с учетом априорного шанса с поправкой Лапласа, значения переменной класса задать как 1, если положительный класс, 0 иначе; для всех количественных данных произвести дискретизацию с равной частотой, в качестве границ интервалов взять выборочные квантили порядка 0, 0.2, 0.4, 0.6, 0.8, 1) (Если какие-то из этих квантилей совпадают, то оставляются только уникальные значения, при этом число интервалов сократится. За исключением крайних, левые границы интервалов – открытые, правые – закрытые).

In [1]:
import numpy as np
import pandas as pd

In [2]:
file = pd.read_csv('V10.csv', sep=';', header=0)

In [3]:
file

,A1,A2,A6,A7,A8,A9,A10,A12,A13,A15,A16
0,b,"33,5",x,h,"4,5",t,t,t,g,857,+
1,a,"23,42",q,v,"1,5",t,t,t,g,400,+
2,b,"20,67",c,v,2,f,f,t,s,0,-
3,a,"28,58",i,bb,"0,5",t,f,t,g,0,-
4,b,"31,83",m,v,"0,04",f,f,f,g,0,-
...,...,...,...,...,...,...,...,...,...,...,...
495,b,"49,58",ff,ff,0,t,t,f,g,0,-
496,a,"22,25",ff,ff,"3,25",f,f,f,g,0,-
497,b,"41,17",cc,h,7,t,t,f,g,0,+
498,b,"22,83",m,v,"1,29",t,t,f,g,800,+


Таблица содержит следующие данные

In [4]:
a1 = ['b', 'a']
a6 = ['c', 'd', 'cc', 'i', 'j', 'k', 'm', 'r', 'q', 'w', 'x', 'e', 'aa', 'ff']
a7 = ['v', 'h', 'bb', 'j', 'n', 'z', 'dd', 'ff', 'o']
a9 = ['t', 'f']
a10 = ['t', 'f']
a12 = ['t', 'f']
a13 = ['g', 'p', 's']

In [5]:
tab = file.copy()

A16 - переменная класса (“+” – положительный класс, “-” – отрицательный)

In [6]:
tab['A16'] = [1 if i == '+' else 0 for i in tab['A16']]

In [7]:
n_plus = list(tab['A16']).count(1)
n_minus = list(tab['A16']).count(0)

In [8]:
n_minus # Абсолютная частота отрицательного класса

274

In [9]:
n_plus # Абсолютная частота положительного класса

226

In [10]:
c = n_plus / n_minus
c # Априорный шанс положительного класса

0.8248175182481752

Калибровка данных с учетом априорного шанса (с поправкой Лапласа)

In [11]:
atr = ['A1', 'A6', 'A7', 'A9', 'A10', 'A12', 'A13']
for a in atr:
    atr_k = []
    for i in tab[a]:
        # Тут все по формуле из отчета
        i_plus = tab[(tab[a] == i) & (tab['A16'] == 1)].shape[0]
        i_minus = tab[(tab[a] == i) & (tab['A16'] == 0)].shape[0]
        atr_k.append(round((i_plus + 1) / (i_plus + 1 + c * (i_minus + 1)), 4))
    tab[a] = atr_k

In [12]:
tab

,A1,A2,A6,A7,A8,A9,A10,A12,A13,A15,A16
0,0.5031,"33,5",0.8359,0.7023,"4,5",0.8115,0.7286,0.5397,0.5153,857,1
1,0.4945,"23,42",0.6783,0.4591,"1,5",0.8115,0.7286,0.5397,0.5153,400,1
2,0.5031,"20,67",0.4980,0.4591,2,0.0777,0.3005,0.5397,0.3371,0,0
3,0.4945,"28,58",0.3125,0.5046,"0,5",0.8115,0.3005,0.5397,0.5153,0,0
4,0.5031,"31,83",0.5026,0.4591,"0,04",0.0777,0.3005,0.4647,0.5153,0,0
...,...,...,...,...,...,...,...,...,...,...,...
495,0.5031,"49,58",0.1513,0.1681,0,0.8115,0.7286,0.4647,0.5153,0,0
496,0.4945,"22,25",0.1513,0.1681,"3,25",0.0777,0.3005,0.4647,0.5153,0,0
497,0.5031,"41,17",0.8124,0.7023,7,0.8115,0.7286,0.4647,0.5153,0,1
498,0.5031,"22,83",0.5026,0.4591,"1,29",0.8115,0.7286,0.4647,0.5153,800,1


In [13]:
new_tab = tab.copy()

Дискретизация с равной частотой. Границами являются выборочные квантили порядка 0, 0.2, 0.4, 0.6, 0.8, 1

In [14]:
atr = ['A2', 'A8', 'A15']
for a in atr:
    atr_k = []
    discr_data = [] # Для хранения дискретизированных значений

    # Преобразуем данные в числа т.к. на данном этапе они являются строками. Ну и меняем , на . для того чтобы преобразовалось без проблем
    numeric_data = [float(str(x).replace(',', '.')) for x in new_tab[a]]
    
    for i in numeric_data:
        # Вычисляются квантили для столбца. Берем только уникальные значения
        quantiles = np.unique(np.quantile(numeric_data, [0, 0.2, 0.4, 0.6, 0.8, 1]))
        # Если попадает в первый интервал
        if i >= quantiles[0] and i <= quantiles[1]:
            discr_data.append(1)
        # Если попадает в другие интервалы (отличие в границах интервалов)
        for j in range(1, len(quantiles) - 1):
            if i > quantiles[j] and i <= quantiles[j + 1]:
                discr_data.append(j + 1)
    new_tab[a] = discr_data

In [15]:
new_tab

,A1,A2,A6,A7,A8,A9,A10,A12,A13,A15,A16
0,0.5031,4,0.8359,0.7023,5,0.8115,0.7286,0.5397,0.5153,3,1
1,0.4945,2,0.6783,0.4591,3,0.8115,0.7286,0.5397,0.5153,2,1
2,0.5031,1,0.4980,0.4591,4,0.0777,0.3005,0.5397,0.3371,1,0
3,0.4945,3,0.3125,0.5046,2,0.8115,0.3005,0.5397,0.5153,1,0
4,0.5031,3,0.5026,0.4591,1,0.0777,0.3005,0.4647,0.5153,1,0
...,...,...,...,...,...,...,...,...,...,...,...
495,0.5031,5,0.1513,0.1681,1,0.8115,0.7286,0.4647,0.5153,1,0
496,0.4945,2,0.1513,0.1681,4,0.0777,0.3005,0.4647,0.5153,1,0
497,0.5031,5,0.8124,0.7023,5,0.8115,0.7286,0.4647,0.5153,1,1
498,0.5031,2,0.5026,0.4591,3,0.8115,0.7286,0.4647,0.5153,3,1


In [16]:
atr = ['A1', 'A2', 'A6', 'A7', 'A8', 'A9', 'A10', 'A12', 'A13', 'A15', 'A16']

In [17]:
chi_matr = pd.DataFrame(index=atr, columns=atr)  # Хи квадрат. Чихал он на нас хи-хи
P_matr = pd.DataFrame(index=atr, columns=atr)  # Коэффициент сопряженности Пирсона
C_matr = pd.DataFrame(index=atr, columns=atr)  # Коэффициент Крамера

In [18]:
from scipy.stats import chi2

In [19]:
chi2_matr = pd.DataFrame(index=atr, columns=atr)  # Таблица отвергается/не отвергается хи-квадрат

Рассчет всех парных показателей взаимосвязи между переменными из набора данных. Были использованы статистика хи-квадрат, коэффициент сопряженности Пирсона, коэффициент Крамера.

In [20]:
for i in range(len(atr)):
    for j in range(i + 1, len(atr)):
        a1 = atr[i]
        a2 = atr[j]

        # Для каждой пары столбцов извлекаем уникальные значения и сортируем 
        px = sorted(new_tab[a1].unique()) 
        py = sorted(new_tab[a2].unique())

        # Подсчет частот совместного появления значений из рассматриваемой пары столбцов
        n = np.zeros((len(px), len(py)))
        for xi in range(len(px)):
            for yi in range(len(py)):
                n[xi, yi] = len(new_tab[(new_tab[a1] == px[xi]) & (new_tab[a2] == py[yi])])

        # Посдсчет статистики хи-квадрат. n_Hat - ожидаемая частота
        res = 0
        for xi in range(len(px)):
            for yi in range(len(py)):
                n_Hat = np.sum(n[xi, :]) * np.sum(n[:, yi]) / 500
                res += ((n[xi, yi] - n_Hat) ** 2) / n_Hat
        chi_matr.at[a1, a2] = res
        chi_matr.at[a2, a1] = res # Матрица симметричная
        
        # Коэфиициент Пирсона для каждой пары столбцов
        P_matr.at[a1, a2] = P_matr.at[a2, a1] = np.sqrt(res / (500 + res))

        # Коэффициент Крамера для каждой пары столбцов
        C_matr.at[a1, a2] = C_matr.at[a2, a1] = np.sqrt(res / (500 * min(len(px) - 1, len(py) - 1)))

        # Рассчет числа степеней свободы
        df = (len(px) - 1) * (len(py) - 1)

        # Нахождение критического значения хи-квадрат
        critical_value = chi2.ppf(1 - 0.05, df)

        # Нулевая гипотеза о независимости отвергается на 95%-м доверительном уровне
        if res > critical_value:
            chi2_matr.at[a1, a2] = chi2_matr.at[a2, a1] = 'Отвергается'
        else:
            chi2_matr.at[a1, a2] = chi2_matr.at[a2, a1] = 'Не отвергается'

In [21]:
chi_matr.to_csv('chi_matrix.csv', sep=';', index=True, encoding='utf-8')
P_matr.to_csv('P_matrix.csv', sep=';', index=True, encoding='utf-8')
C_matr.to_csv('C_matrix.csv', sep=';', index=True, encoding='utf-8')

In [22]:
chi2_matr.to_csv('chi2_matrix.csv', sep=';', index=True, encoding='cp1251')

In [23]:
matr_p = pd.DataFrame(index=atr, columns=atr)

Проверка гипотезы о значимости взаимосвязей между переменными по критерию хи-квадрат и с помощью перестановочного критерия. Количество перестановок М = 200

In [24]:
M = 200
for i in range(len(atr)):
    for j in range(i + 1, len(atr)):
        a1 = atr[i]
        a2 = atr[j]

        # Для хранения результатов хи-квадрат теста каждой перестановки
        chis = []
        for m in range(M):
            px = sorted(new_tab[a1].unique())
            py = sorted(new_tab[a2].unique())
            pdx = pd.DataFrame(np.random.permutation(new_tab[a1]), columns=[a1])
            n = np.zeros((len(px), len(py)))
            for xi in range(len(px)):
                for yi in range(len(py)):
                    n[xi, yi] = len(new_tab[(pdx[a1] == px[xi]) & (new_tab[a2] == py[yi])])
            res = 0
            for xi in range(len(px)):
                for yi in range(len(py)):
                    n_t = np.sum(n[xi, :]) * np.sum(n[:, yi]) / 500
                    res += ((n[xi, yi] - n_t) ** 2) / n_t
            chis.append(res)
        
        # После выполнения всех перестановок вычисляется 95-й квантиль списка
        quantile = np.quantile(chis, 0.95)
        matr_p.at[a1, a2] = matr_p.at[a2, a1] = quantile

In [25]:
matr_p.to_csv('matr_p_matrix.csv', sep=';', index=True, encoding='utf-8')

In [26]:
matr_diff = pd.DataFrame(index=atr, columns=atr)
# Если разница между новой матрицей статистики хи-квадрат и рассчитанной ранее статистикой хи-квадрат меньше нуля, то гипотеза отвергается
# Иначе она принимается
# Это позволяет определить, насколько статистически значима связь между столбцами
for i in range(len(atr)):
    for j in range(i + 1, len(atr)):
        a1 = atr[i]
        a2 = atr[j]
        if (matr_p.at[a1, a2] - chi_matr.at[a1, a2] < 0):
            matr_diff.at[a1, a2] = matr_diff.at[a2, a1] = 'Отвергается'
        else:
            matr_diff.at[a1, a2] = matr_diff.at[a2, a1] = 'Не отвергается'

matr_diff.to_csv('matr_diff.csv', sep=';', index=True, encoding='cp1251')

Показатель частной взаимной информации. Из подмножества признаков, значимо влияющих на переменную класса, была выберана пара признаков, наиболее сильно взаимосвязанных друг с другом. Обозначим их X1 и X2. Проверить наличие частной корреляции между признаком X1 (X2) и откликом при исключении влияния X2 (X1) с помощью показателя частной взаимной информации.

In [27]:
from collections import Counter

def calculate_probabilities(data):
    """
    Вычисляет вероятности для данных.

    """
    counter = Counter(data)
    total = len(data)
    probabilities = {k: v / total for k, v in counter.items()}
    return probabilities

def conditional_entropy(X, Y, Z=None):
    """
    Вычисляет условную энтропию H(X1|Y, X2) или H(X1|Y).

    X: Данные для случайной величины X1.
    Y: Данные для случайной величины Y.
    Z: Данные для случайной величины X2. По умолчанию None.

    """
    if Z is None:
        # Вычисление условной энтропии H(X|Y)
        data = list(zip(X, Y))
        P_xz = calculate_probabilities(data)
        data_z = list(Y)
        P_z = calculate_probabilities(data_z)

        X_values = np.unique(X)
        Z_values = np.unique(Y)

        P_xz_array = np.zeros((len(X_values), len(Z_values)))
        P_z_array = np.zeros(len(Z_values))

        for x, z in data:
            i = np.where(X_values == x)[0][0]
            j = np.where(Z_values == z)[0][0]
            P_xz_array[i, j] = P_xz[(x, z)]

        for z in data_z:
            j = np.where(Z_values == z)[0][0]
            P_z_array[j] = P_z[z]

        H_x_given_z = 0
        for i in range(len(X_values)):
            for j in range(len(Z_values)):
                if P_z_array[j] != 0:
                    P_x_given_z = P_xz_array[i, j] / P_z_array[j]
                    if P_x_given_z != 0:
                        H_x_given_z -= P_xz_array[i, j] * np.log2(P_x_given_z)

        return H_x_given_z
    else:
        # Вычисление условной энтропии H(X1|Y, Х2)
        data = list(zip(X, Y, Z))
        P_xyz = calculate_probabilities(data)
        data_yz = list(zip(Y, Z))
        P_yz = calculate_probabilities(data_yz)

        X_values = np.unique(X)
        Y_values = np.unique(Y)
        Z_values = np.unique(Z)

        P_xyz_array = np.zeros((len(X_values), len(Y_values), len(Z_values)))
        P_yz_array = np.zeros((len(Y_values), len(Z_values)))

        for x, y, z in data:
            i = np.where(X_values == x)[0][0]
            j = np.where(Y_values == y)[0][0]
            k = np.where(Z_values == z)[0][0]
            P_xyz_array[i, j, k] = P_xyz[(x, y, z)]

        for y, z in data_yz:
            j = np.where(Y_values == y)[0][0]
            k = np.where(Z_values == z)[0][0]
            P_yz_array[j, k] = P_yz[(y, z)]

        H_x_given_yz = 0
        for i in range(len(X_values)):
            for j in range(len(Y_values)):
                for k in range(len(Z_values)):
                    if P_yz_array[j, k] != 0:
                        P_x_given_yz = P_xyz_array[i, j, k] / P_yz_array[j, k]
                        if P_x_given_yz != 0:
                            H_x_given_yz -= P_xyz_array[i, j, k] * np.log2(P_x_given_yz)

        return H_x_given_yz
    
def partial_mutual_information(X, Y, Z):
    """
    Вычисляет частную взаимную информацию ЧВИ(X1, Y | Х2).

    X: Данные для случайной величины X1.
    Y: Данные для случайной величины Y.
    Z: Данные для случайной величины Х2.

    """
    H_X_given_Z = conditional_entropy(X, Z)
    H_XY_given_Z = conditional_entropy(X, Y, Z)

    PMI = H_X_given_Z - H_XY_given_Z
    return PMI

def permutation_test(X, Y, Z, n_permutations=1000):
    """
    Проверяет статистическую значимость ЧВИ с использованием перестановочного критерия.

    X: Данные для случайной величины X1.
    Y: Данные для случайной величины Y.
    Z: Данные для случайной величины X2.
    n_permutations: Количество перестановок для перестановочного критерия.

    """
    observed_pmi = partial_mutual_information(X, Y, Z)
    permutation_pmi_values = []

    for _ in range(n_permutations):
        permuted_Z = np.random.permutation(Z)
        permuted_pmi = partial_mutual_information(X, Y, permuted_Z)
        permutation_pmi_values.append(permuted_pmi)

    # Вычисление p-значения
    # permutation_pmi_values - список значений ЧВИ, вычисленных для каждой перестановки переменной X2
    # observed_pmi - ЧВИ исходных данных 
    # Считаем сколько раз в переставленных выборках мы получили значение ЧВИ, которое по крайней мере такое же большое, как наблюдаемое значение
    # Добавляем 1 для учета наблюдаемого значения ЧВИ в исходных данных
    # Вычисляет p-значение как отношение количества переставленных значений ЧВИ, которые больше или равны наблюдаемому значению
    p_value = (np.sum(np.array(permutation_pmi_values) >= observed_pmi) + 1) / (n_permutations + 1)
    return p_value

In [28]:
atr = ['A6', 'A7', 'A16'] # Выбранная пара признаков и переменная класса

In [29]:
a1 = atr[0] 
a2 = atr[1] 
a3 = atr[2] 

X1 = new_tab[a1].values  # Признак X1
X2 = new_tab[a2].values  # Признак X2
Y = new_tab[a3].values # Переменная класса

# Вычисление ЧВИ(X1, Y | X2)
pmi = partial_mutual_information(X1, Y, X2)
print(f"Частная взаимная информация ЧВИ(X1, Y | X2): {pmi}")

# Проверка статистической значимости ЧВИ с использованием перестановочного критерия
p_value = permutation_test(X1, Y, X2, n_permutations=500)

alpha = 0.05
if p_value < alpha:
    print("ЧВИ статистически значимо, существует частная корреляция между X1 и Y при условии X2.")
else:
    print("ЧВИ не является статистически значимым, нет частной корреляции между X1 и Y при условии X2.")

Частная взаимная информация ЧВИ(X1, Y | X2): 0.13411205095626855
ЧВИ не является статистически значимым, нет частной корреляции между X1 и Y при условии X2.
